In [1]:
# ============================================================================
# Section 1: CẤU HÌNH & IMPORT
# ============================================================================
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import gc
import warnings
!pip install goatools

# Tắt các cảnh báo không cần thiết
warnings.filterwarnings('ignore')

# --- CẤU HÌNH ĐƯỜNG DẪN (Chỉ dùng T5) ---
# Hãy đảm bảo đường dẫn này đúng với dataset T5 bạn đã add
EMBED_DIR = '/kaggle/input/cafa-6-t5-embeddings' 
TRAIN_TERMS_PATH = '/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv'
OBO_PATH = '/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo'

# --- HYPERPARAMETERS (TỐI ƯU HÓA) ---
BATCH_SIZE = 512       # Batch lớn giúp train nhanh và ổn định gradient
EPOCHS = 25           # Tăng epoch vì dùng Cosine Scheduler
LEARNING_RATE = 1e-3   # Tốc độ học khởi điểm
NUM_LABELS = 1500      # Số lượng nhãn GO phổ biến nhất
SEEDS = [42, 2024, 123, 777, 888] # Ensemble 5 mô hình để kết quả vững chắc nhất

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔧 Device: {device}")
print("🚀 Strategy: T5 Embeddings Only + MLP Multi-label")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 11.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 11.9 MB/s eta 0:00:00
🔧 Device: cuda
🚀 Strategy: T5 Embeddings Only + MLP Multi-label


In [2]:
# ============================================================================
# Section 2: LOAD DỮ LIỆU & CHUẨN HÓA (STANDARD SCALER)
# ============================================================================
print("\n" + "="*60)
print("📂 BƯỚC 1: LOAD DỮ LIỆU T5 & CHUẨN HÓA")
print("="*60)

# Hàm làm sạch ID để khớp giữa các file
def clean_id(pid):
    if isinstance(pid, bytes): pid = pid.decode('utf-8')
    pid = str(pid).strip()
    if '|' in pid: 
        parts = pid.split('|')
        return parts[1] if len(parts) >= 2 else pid
    return pid
clean_func = np.vectorize(clean_id)

# 1. Load T5 Embeddings (Train)
print("1️⃣ Loading T5 Train Embeddings...")
try:
    X_raw = np.load(os.path.join(EMBED_DIR, 'train_embeds.npy'))
    ids = clean_func(np.load(os.path.join(EMBED_DIR, 'train_ids.npy')))
    print(f"   ✓ Loaded Shape: {X_raw.shape}")
except Exception as e:
    print(f"❌ Error loading T5: {e}"); exit()

# 2. Load Labels
print("2️⃣ Loading Labels...")
df_terms = pd.read_csv(TRAIN_TERMS_PATH, sep='\t')
if 'EntryID' not in df_terms.columns:
    df_terms = pd.read_csv(TRAIN_TERMS_PATH, sep='\t', header=None, names=['EntryID', 'term', 'aspect'])
df_terms['EntryID'] = df_terms['EntryID'].apply(clean_id)

# 3. Align Data (Tìm ID chung)
print("3️⃣ Aligning Data...")
common_ids = sorted(list(set(ids) & set(df_terms['EntryID'])))
print(f"   ✓ Số lượng protein dùng để train: {len(common_ids)}")

if len(common_ids) == 0: raise ValueError("❌ Không tìm thấy ID chung!")

# Map ID -> Index
id_map = {pid: i for i, pid in enumerate(ids)}
final_map = {pid: i for i, pid in enumerate(common_ids)}

# Tạo ma trận X, y
INPUT_DIM = X_raw.shape[1]
X_all = np.zeros((len(common_ids), INPUT_DIM), dtype=np.float32)
y_all = np.zeros((len(common_ids), NUM_LABELS), dtype=np.float32)

print("   -> Building Feature Matrix X...")
for i, pid in enumerate(common_ids):
    X_all[i] = X_raw[id_map[pid]]

print("   -> Building Label Matrix y (Multi-label)...")
top_terms = df_terms['term'].value_counts().head(NUM_LABELS).index.tolist()
term2idx = {t: i for i, t in enumerate(top_terms)}
idx2term = {i: t for i, t in enumerate(top_terms)}

relevant = df_terms[df_terms['EntryID'].isin(common_ids) & df_terms['term'].isin(top_terms)]
for row in relevant.itertuples(index=False):
    pid = getattr(row, 'EntryID', row[0])
    term = getattr(row, 'term', row[1])
    if pid in final_map and term in term2idx:
        y_all[final_map[pid], term2idx[term]] = 1.0

# 4. CHUẨN HÓA DỮ LIỆU (QUAN TRỌNG)
print("4️⃣ Applying StandardScaler...")
scaler = StandardScaler()
X_all = scaler.fit_transform(X_all) # Đưa về Mean=0, Std=1
print("   ✓ Done! Dữ liệu đã được chuẩn hóa.")

# Dọn dẹp RAM
del X_raw, ids, df_terms, relevant
gc.collect()

print(f"✅ DATA READY: X={X_all.shape}, y={y_all.shape}")


📂 BƯỚC 1: LOAD DỮ LIỆU T5 & CHUẨN HÓA
1️⃣ Loading T5 Train Embeddings...
   ✓ Loaded Shape: (82404, 1024)
2️⃣ Loading Labels...
3️⃣ Aligning Data...
   ✓ Số lượng protein dùng để train: 82404
   -> Building Feature Matrix X...
   -> Building Label Matrix y (Multi-label)...
4️⃣ Applying StandardScaler...
   ✓ Done! Dữ liệu đã được chuẩn hóa.
✅ DATA READY: X=(82404, 1024), y=(82404, 1500)


In [3]:
# ============================================================================
# Section 3: MODEL (RESIDUAL MLP) & DATASET (REVERTED TO STANDARD)
# ============================================================================

class TensorDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32) if y is not None else None
    def __len__(self): return len(self.X)
    def __getitem__(self, idx):
        if self.y is not None: return self.X[idx], self.y[idx]
        return self.X[idx]

class ResidualBlock(nn.Module):
    def __init__(self, dim, dropout=0.4): 
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, dim),
            nn.BatchNorm1d(dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
    def forward(self, x): return x + self.net(x)

class ResMLP(nn.Module):
    def __init__(self, inp_dim, out_dim):
        super().__init__()
        self.entry = nn.Sequential(
            nn.Linear(inp_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.35)
        )
        self.blocks = nn.Sequential(
            ResidualBlock(1024),
            ResidualBlock(1024)
        )
        self.head = nn.Linear(1024, out_dim)
    
    def forward(self, x):
        x = self.entry(x)
        x = self.blocks(x)
        return self.head(x)

In [4]:
# ============================================================================
# Section 4: TRAINING FUNCTION (REVERTED TO BCE)
# ============================================================================

def train_single_seed(seed, X_tr, y_tr, X_va, y_va, save_path):
    print(f"\n🎲 Training Seed: {seed}")
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    train_loader = DataLoader(TensorDataset(X_tr, y_tr), batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_va, y_va), batch_size=BATCH_SIZE, shuffle=False)
    
    model = ResMLP(INPUT_DIM, NUM_LABELS).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
    
    # --- QUAY VỀ STANDARD BCE ---
    # Đây là hàm loss ổn định nhất cho baseline 0.348
    criterion = nn.BCEWithLogitsLoss()
    
    # CosineAnnealing giúp learning rate giảm mượt mà
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
    
    best_loss = float('inf')
    
    for ep in range(EPOCHS):
        model.train()
        t_loss = 0
        for fx, fy in train_loader:
            fx, fy = fx.to(device), fy.to(device)
            optimizer.zero_grad()
            out = model(fx)
            loss = criterion(out, fy) # BCE dùng Logits trực tiếp, không qua Sigmoid
            loss.backward()
            optimizer.step()
            t_loss += loss.item()
            
        model.eval()
        v_loss = 0
        with torch.no_grad():
            for fx, fy in val_loader:
                # Validation cũng dùng BCEWithLogitsLoss
                v_loss += criterion(model(fx.to(device)), fy.to(device)).item()
        
        avg_v = v_loss/len(val_loader)
        
        if avg_v < best_loss:
            best_loss = avg_v
            torch.save(model.state_dict(), save_path)
            
        scheduler.step()
        
        
        lr_curr = optimizer.param_groups[0]['lr']
        print(f"   Ep {ep+1}/{EPOCHS}: Train={t_loss/len(train_loader):.4f} | Val={avg_v:.4f} | LR={lr_curr:.6f}")
            
    return best_loss

In [5]:
# ============================================================================
# Section 5: CHẠY TRAINING (ENSEMBLE 5 SEEDS)
# ============================================================================
print("\n" + "="*60)
print("🚀 BƯỚC 2: TRAIN ENSEMBLE MODELS")
print("="*60)

os.makedirs('models_t5_final', exist_ok=True)
model_paths = []

for seed in SEEDS:
    X_tr, X_va, y_tr, y_va = train_test_split(X_all, y_all, test_size=0.1, random_state=seed)
    path = f'models_t5_final/model_{seed}.pth'
    train_single_seed(seed, X_tr, y_tr, X_va, y_va, path)
    model_paths.append(path)
    
    del X_tr, X_va, y_tr, y_va
    gc.collect()
    torch.cuda.empty_cache()
    


🚀 BƯỚC 2: TRAIN ENSEMBLE MODELS

🎲 Training Seed: 42
   Ep 1/25: Train=0.0251 | Val=0.0134 | LR=0.000996
   Ep 2/25: Train=0.0129 | Val=0.0122 | LR=0.000984
   Ep 3/25: Train=0.0120 | Val=0.0117 | LR=0.000965
   Ep 4/25: Train=0.0115 | Val=0.0114 | LR=0.000938
   Ep 5/25: Train=0.0112 | Val=0.0112 | LR=0.000905
   Ep 6/25: Train=0.0109 | Val=0.0112 | LR=0.000865
   Ep 7/25: Train=0.0107 | Val=0.0111 | LR=0.000819
   Ep 8/25: Train=0.0106 | Val=0.0110 | LR=0.000768
   Ep 9/25: Train=0.0104 | Val=0.0109 | LR=0.000713
   Ep 10/25: Train=0.0102 | Val=0.0109 | LR=0.000655
   Ep 11/25: Train=0.0101 | Val=0.0109 | LR=0.000594
   Ep 12/25: Train=0.0099 | Val=0.0108 | LR=0.000532
   Ep 13/25: Train=0.0098 | Val=0.0108 | LR=0.000469
   Ep 14/25: Train=0.0097 | Val=0.0108 | LR=0.000407
   Ep 15/25: Train=0.0095 | Val=0.0107 | LR=0.000346
   Ep 16/25: Train=0.0094 | Val=0.0107 | LR=0.000288
   Ep 17/25: Train=0.0093 | Val=0.0107 | LR=0.000233
   Ep 18/25: Train=0.0092 | Val=0.0107 | LR=0.000182
 

In [6]:
# ============================================================================
# Section 6: DỰ ĐOÁN & HẬU XỬ LÝ (TÍCH HỢP GOA & PROPAGATION)
# ============================================================================
import pandas as pd
from tqdm import tqdm

print("\n" + "="*60)
print("🔮 BƯỚC 3: DỰ ĐOÁN, XỬ LÝ GOA & PROPAGATION")
print("="*60)

# --- CẤU HÌNH ĐƯỜNG DẪN MỚI ---
# File GOA bạn đã upload
GOA_PATH = '/kaggle/input/protein-go-annotations/goa_uniprot_all.csv'
TEMP_FILE = 'submission_raw.tsv'
FINAL_OUTPUT = 'submission.tsv'

# ---------------------------------------------------------
# 1. CHUẨN BỊ DỮ LIỆU TEST
# ---------------------------------------------------------
print("1️⃣ Loading & Normalizing Test Data...")

try:
    test_emb = np.load(os.path.join(EMBED_DIR, 'test_embeds.npy'))
    test_ids = clean_func(np.load(os.path.join(EMBED_DIR, 'test_ids.npy')))
    test_X = scaler.transform(test_emb) # Dùng scaler của tập train
    print(f"   ✓ Test Shape: {test_X.shape}")
except Exception as e:
    print(f"❌ Error loading test data: {e}"); exit()

# ---------------------------------------------------------
# 2. DỰ ĐOÁN (ENSEMBLE)
# ---------------------------------------------------------
print("2️⃣ Predicting with Ensemble Models...")
models = [ResMLP(INPUT_DIM, NUM_LABELS).to(device) for _ in model_paths]
for i, m in enumerate(models):
    m.load_state_dict(torch.load(model_paths[i]))
    m.eval()

loader = DataLoader(TensorDataset(test_X), batch_size=BATCH_SIZE*2, shuffle=False)

# Ghi file thô để tiết kiệm RAM
with open(TEMP_FILE, 'w') as f:
    start_idx = 0
    with torch.no_grad():
        for fx in tqdm(loader, desc="Inference"):
            fx = fx.to(device)
            bs = fx.size(0)
            
            # Tính trung bình cộng xác suất
            avg_prob = torch.zeros((bs, NUM_LABELS)).to(device)
            for m in models:
                avg_prob += torch.sigmoid(m(fx))
            avg_prob /= len(models)
            avg_prob = avg_prob.cpu().numpy()
            
            # Ghi ra file
            for k in range(bs):
                pid = test_ids[start_idx + k]
                probs = avg_prob[k]
                # Lấy Top 80 để dư dả cho bước filter
                top_k = np.argsort(probs)[::-1][:80]
                
                for idx in top_k:
                    sc = probs[idx]
                    if sc > 0.001: 
                        f.write(f"{pid}\t{idx2term[idx]}\t{sc:.4f}\n")
            start_idx += bs

print(f"   ✓ Raw predictions saved to {TEMP_FILE}")
del test_X, models, loader
gc.collect()

# ---------------------------------------------------------
# 3. HẬU XỬ LÝ: GOA KNOWLEDGE & PROPAGATION
# ---------------------------------------------------------
print("3️⃣ Applying Advanced Post-Processing...")

# --- Hàm hỗ trợ ---
def get_descendants(term, children_map, cache):
    if term in cache: return cache[term]
    res = set()
    if term in children_map:
        for child in children_map[term]:
            res.add(child)
            res.update(get_descendants(child, children_map, cache))
    cache[term] = res
    return res

try:
    from goatools.obo_parser import GODag
    if os.path.exists(OBO_PATH):
        # A. Xây dựng cây OBO
        print("   -> Building GO Hierarchy...")
        godag = GODag(OBO_PATH)
        go_children = {tid: [c.id for c in term.children] for tid, term in godag.items()}
        
        # B. Load GOA để lấy Negative & Ground Truth
        print(f"   -> Loading External Knowledge from {GOA_PATH}...")
        neg_keys = set()
        pos_df = None
        
        if os.path.exists(GOA_PATH):
            goa_df = pd.read_csv(GOA_PATH)
            # Chuẩn hóa tên cột
            if 'DB_Object_ID' in goa_df.columns: goa_df.rename(columns={'DB_Object_ID':'protein_id'}, inplace=True)
            if 'GO_ID' in goa_df.columns: goa_df.rename(columns={'GO_ID':'go_term'}, inplace=True)
            if 'Qualifier' in goa_df.columns: goa_df.rename(columns={'Qualifier':'qualifier'}, inplace=True)
            
            # 1. Negative Logic
            if 'qualifier' in goa_df.columns:
                neg = goa_df[goa_df['qualifier'].str.contains('NOT', na=False)]
                if not neg.empty:
                    desc_cache = {}
                    print("      Proagating Negative Constraints...")
                    for pid, grp in neg.groupby('protein_id'):
                        terms = grp['go_term'].tolist()
                        all_neg = set(terms)
                        for t in terms:
                            all_neg |= get_descendants(t, go_children, desc_cache)
                        for t in all_neg:
                            neg_keys.add(f"{pid}_{t}")
                            
            # 2. Positive Logic (Ground Truth)
            pos_df = goa_df[~goa_df['qualifier'].str.contains('NOT', na=False)][['protein_id', 'go_term']].copy()
            pos_df['Score'] = 1.0
            pos_df.rename(columns={'protein_id':'ProteinID', 'go_term':'Term'}, inplace=True)
            # Lọc mâu thuẫn
            pos_df['key'] = pos_df['ProteinID'].astype(str) + '_' + pos_df['Term'].astype(str)
            pos_df = pos_df[~pos_df['key'].isin(neg_keys)].drop(columns=['key']).drop_duplicates()
        
        # C. Xử lý File Dự Đoán
        print("   -> Processing Predictions...")
        # Đọc file raw
        df = pd.read_csv(TEMP_FILE, sep='\t', names=['ProteinID', 'Term', 'Score'], dtype={'ProteinID': str})
        
        # 1. Negative Filtering: Xóa dòng vi phạm
        if neg_keys:
            df['key'] = df['ProteinID'].astype(str) + '_' + df['Term'].astype(str)
            init_len = len(df)
            df = df[~df['key'].isin(neg_keys)].drop(columns=['key'])
            print(f"      Removed {init_len - len(df)} negative predictions.")
            
        # 2. Ground Truth Injection: Thêm nhãn đúng
        if pos_df is not None and not pos_df.empty:
            test_prots = set(df['ProteinID'].unique())
            relevant_gt = pos_df[pos_df['ProteinID'].isin(test_prots)]
            df = pd.concat([df, relevant_gt], ignore_index=True)
            # Lấy max score nếu trùng
            df = df.groupby(['ProteinID', 'Term'], as_index=False)['Score'].max()
            print(f"      Injected Ground Truth data.")

        # D. Propagation (Lan truyền điểm số lên cha)
        # Logic: Điểm của cha = max(Điểm cha dự đoán, Điểm các con)
        print("   -> Running Hierarchy Propagation...")
        final_rows = []
        
        # Hàm propagate tối ưu hơn
        for pid, grp in tqdm(df.groupby('ProteinID'), desc="Propagating"):
            scores = dict(zip(grp['Term'], grp['Score']))
            # Chỉ lan truyền những term có trong OBO
            terms_to_check = [t for t in scores.keys() if t in godag]
            
            for term in terms_to_check:
                current_score = scores[term]
                # Lan truyền lên tất cả cha (tổ tiên)
                parents = godag[term].get_all_parents()
                for p in parents:
                    scores[p] = max(scores.get(p, 0), current_score)
            
            # Lọc threshold cuối cùng
            final_rows.extend([[pid, t, s] for t, s in scores.items() if s >= 0.01])

        # E. Lưu File Cuối
        res_df = pd.DataFrame(final_rows, columns=['ProteinID', 'Term', 'Score'])
        res_df['Score'] = res_df['Score'].apply(lambda x: f"{x:.3f}")
        
        # Sắp xếp nhẹ để đẹp file (không bắt buộc nhưng tốt)
        res_df.sort_values(by=['ProteinID', 'Score'], ascending=[True, False], inplace=True)
        
        res_df.to_csv(FINAL_OUTPUT, sep='\t', index=False, header=False)
        print(f"✅ DONE! Final Submission Saved: {FINAL_OUTPUT}")

    else:
        print("⚠️ OBO file missing. Renaming raw file.")
        os.rename(TEMP_FILE, FINAL_OUTPUT)

except Exception as e:
    print(f"❌ Critical Error in Post-processing: {e}")
    # Fallback: Nếu lỗi thì nộp file raw
    if os.path.exists(TEMP_FILE):
        print("   -> Fallback to raw submission.")
        if os.path.exists(FINAL_OUTPUT): os.remove(FINAL_OUTPUT)
        os.rename(TEMP_FILE, FINAL_OUTPUT)


🔮 BƯỚC 3: DỰ ĐOÁN, XỬ LÝ GOA & PROPAGATION
1️⃣ Loading & Normalizing Test Data...
   ✓ Test Shape: (224309, 1024)
2️⃣ Predicting with Ensemble Models...


Inference: 100%|██████████| 220/220 [01:04<00:00,  3.44it/s]


   ✓ Raw predictions saved to submission_raw.tsv
3️⃣ Applying Advanced Post-Processing...
   -> Building GO Hierarchy...
/kaggle/input/cafa-6-protein-function-prediction/Train/go-basic.obo: fmt(1.2) rel(2025-06-01) 43,448 Terms
   -> Loading External Knowledge from /kaggle/input/protein-go-annotations/goa_uniprot_all.csv...
      Proagating Negative Constraints...
   -> Processing Predictions...
      Removed 3145 negative predictions.
      Injected Ground Truth data.
   -> Running Hierarchy Propagation...


Propagating: 100%|██████████| 224309/224309 [03:09<00:00, 1185.48it/s]


✅ DONE! Final Submission Saved: submission.tsv
